In [ ]:
import sys
!{sys.executable} -m pip install numpy

In [1]:
from numpy import random

# Predicting season records

These are very simple models for predicting a team's full season record based on runs scored (RS) and runs allowed (RA).

Data (collected below) for RS and RA are from the 2022 MLB season, provided by Baseball-Reference.

In [2]:
# los angeles dodgers
lad_avg_rs = 5.23
lad_avg_ra = 3.17

# san diego padres
sdp_avg_rs = 4.35
sdp_avg_ra = 4.07

# league average
lg_avg_rs = 4.28
lg_avg_ra = 4.28

## Pythagorean win expectancy

This method retrospectively estimates a team's season record based on runs scored and runs allowed using the simple formula: `win% = rs^2 / (rs^2 + ra^2)`

https://fisher.wharton.upenn.edu/wp-content/uploads/2020/09/Thesis_Andrew-Cui.pdf

In [3]:
def pythagorean_win_expectancy(avg_rs, avg_ra):
    win_pct = (avg_rs ** 2) / (avg_rs ** 2 + avg_ra ** 2)
    wins = int(win_pct * 162 + 0.5) # rounded to nearest whole number
    losses = 162 - wins # 162 games per season

    return (win_pct, wins, losses)

In [4]:
print('Pythagorean win expectancy')
print('-' * 20)

lad_win_pct, lad_wins, lad_losses = pythagorean_win_expectancy(lad_avg_rs, lad_avg_ra)
print(f'LAD: {lad_wins}-{lad_losses} ({round(lad_win_pct, 3)})')

sdp_win_pct, sdp_wins, sdp_losses = pythagorean_win_expectancy(sdp_avg_rs, sdp_avg_ra)
print(f'SDP: {sdp_wins}-{sdp_losses} ({round(sdp_win_pct, 3)})')

Pythagorean win expectancy
--------------------
LAD: 118-44 (0.731)
SDP: 86-76 (0.533)


## Poisson distribution

This method utilizes the Poisson distribution to "simulate" every game of the season, assuming every game is against a league average team. For each game, the model will predict a value (from the Poisson distribution) for runs scored and allowed for both teams, take the average, and determine the winner.

https://nejsds.nestat.org/journal/NEJSDS/article/1/read

In [8]:
def poisson_distribution(avg_rs, avg_ra, opp_avg_rs, opp_avg_ra):
    opp_aggr_rs = random.poisson(lam=opp_avg_rs, size=162)
    opp_aggr_ra = random.poisson(lam=opp_avg_ra, size=162)
    
    aggr_rs = random.poisson(lam=avg_rs, size=162)
    aggr_ra = random.poisson(lam=avg_ra, size=162)
    
    wins = 0
    losses = 0
    
    for i in range(0, 162):
        runs_scored = (aggr_rs[i] + opp_aggr_ra[i]) / 2
        runs_allowed = (aggr_ra[i] + opp_aggr_rs[i]) / 2
    
        if runs_scored >= runs_allowed: # what to do with tied scores?
            wins += 1
        else:
            losses += 1
    
    win_pct = wins / 162

    return (win_pct, wins, losses)

In [13]:
print('Poisson distribution')
print('-' * 20)

lad_win_pct, lad_wins, lad_losses = poisson_distribution(lad_avg_rs, lad_avg_ra, lg_avg_rs, lg_avg_ra)
print(f'LAD: {lad_wins}-{lad_losses} ({round(lad_win_pct, 3)})')

sdp_win_pct, sdp_wins, sdp_losses = poisson_distribution(sdp_avg_rs, sdp_avg_ra, lg_avg_rs, lg_avg_ra)
print(f'SDP: {sdp_wins}-{sdp_losses} ({round(sdp_win_pct, 3)})')

Poisson distribution
--------------------
LAD: 116-46 (0.716)
SDP: 91-71 (0.562)
